<a href="https://colab.research.google.com/github/chethanmo/chethanmo.github.io/blob/master/ath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas pandas-ta numpy matplotlib yfinance
!pip install --upgrade pandas-datareader

In [ ]:
import pandas_ta as ta
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import yfinance as yf
from dateutil.relativedelta import relativedelta
from datetime import date as date


In [ ]:
#ticker_yahoo = yf.Ticker('HINDZINC.NS')
# ticker_yahoo.fast_info

file_path = '/content/EQUITY_L.csv'
fields = ['SYMBOL']
tickers = pd.read_csv(file_path, usecols=fields)


In [ ]:
##lazy-loading dict with keys = ['currency', 'exchange', 'timezone', 'shares', 'market_cap', 'last_price', 'previous_close', 'open', 'day_high', 'day_low', 'regular_market_previous_close', 'last_volume', 'fifty_day_average', 'two_hundred_day_average', 'ten_day_average_volume', 'three_month_average_volume', 'year_high', 'year_low', 'year_change']

In [ ]:
from concurrent.futures import ThreadPoolExecutor

ath_list = []
for i in range(0,len(tickers)):
  ath_list.append(tickers['SYMBOL'][i] +'.NS')
  


def get_stats(ticker):
    info = yf.Tickers(ticker).tickers[ticker].fast_info
    #print(f"{ticker} {info['last_price']} {info['year_high']}")
    return ticker, info['last_price'], info['year_high'], info['two_hundred_day_average']
def paralel_func(ticker):
    with ThreadPoolExecutor(max_workers=None) as executor:
        results = list(executor.map(get_stats, ath_list))
    return results

results = paralel_func(ath_list)


In [ ]:
ticker_list = pd.DataFrame(results, columns=['Ticker', 'Last_price', 'ATH', 'two_EMA']) #creating the dataframe

pd.set_option('display.max_rows', None)

ticker_list = ticker_list.query('Last_price < ATH * 0.95 and Last_price > ATH * 0.9 and Last_price > two_EMA')
ticker_list.reset_index(drop=True, inplace=True)

#ticker_list.to_csv('Watchlist.csv', index=False)
#ticker_list


In [ ]:
startdate = date.today() - relativedelta(years=4)
enddate = date.today()

x = ticker_list['Ticker'].to_string(header=False,index=False).split('\n')
tickers = [','.join(ele.split()) for ele in x]

df = yf.download(tickers, start= startdate, end= enddate)


[*********************100%***********************]  104 of 104 completed


In [ ]:
pd.set_option('display.max_rows', 10)

ax_df = df.copy()

df1 = pd.DataFrame()


for ticker in tickers:
  test = ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=10, multiplier=1)
  test1 = ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=11, multiplier=2)
  test2 = ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=12, multiplier=3)
  test3 = ta.ema(close=ax_df[('Close', ticker)], length=20).to_frame()
  test4 = ta.ema(close=ax_df[('High', ticker)], length=20).to_frame()
  test5 = ta.ema(close=ax_df[('Low', ticker)], length=20).to_frame()
  test.columns = pd.MultiIndex.from_product([test.columns, [ticker]])
  test1.columns = pd.MultiIndex.from_product([test1.columns, [ticker]])
  test2.columns = pd.MultiIndex.from_product([test2.columns, [ticker]])
  test3.columns = pd.MultiIndex.from_product([test3.columns, [ticker]])
  test4.columns = pd.MultiIndex.from_product([test4.columns, [ticker]])
  test5.columns = pd.MultiIndex.from_product([test5.columns, [ticker]])
  df1 = pd.concat([df1, test, test1, test2, test3, test4, test5], axis=1)
  df1.sort_index()
  
ax_df = pd.concat([ax_df, df1], axis=1)
ax_df.reset_index()
ax_df = ax_df.drop(['Adj Close','Open','Low','High','Volume'], level=0, axis=1)


/usr/local/lib/python3.8/dist-packages/IPython/core/async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)


In [134]:
ax_df.tail(2)

Close                                       \
                          ABBOTINDIA.NS      ACE.NS AEGISCHEM.NS  AGARIND.NS   
Date                                                                           
2023-02-02 00:00:00+05:30  20480.300781  331.399994   358.649994  635.849976   
2023-02-03 00:00:00+05:30  20872.250000  329.549988   367.200012  641.250000   

                                                                    \
                          AKZOINDIA.NS AMARAJABAT.NS ANANDRATHI.NS   
Date                                                                 
2023-02-02 00:00:00+05:30  2213.899902    615.549988    762.900024   
2023-02-03 00:00:00+05:30  2210.300049    605.049988    763.150024   

                                                                ...  \
                           APARINDS.NS  APTECHT.NS ASHOKLEY.NS  ...   
Date                                                            ...   
2023-02-02 00:00:00+05:30  2015.949951  357.899994  152.050003  ...   
2023-02-03 00:00:00+05:30  1931.099976  362.500000  153.899994  ...   

                          SUPERTd_11_2.0 SUPERTl_11_2.0 SUPERTs_11_2.0  \
                                VINNY.NS       VINNY.NS       VINNY.NS   
Date                                                                     
2023-02-02 00:00:00+05:30              1     233.360073            NaN   
2023-02-03 00:00:00+05:30              1     238.190979            NaN   

                          SUPERT_12_3.0 SUPERTd_12_3.0 SUPERTl_12_3.0  \
                               VINNY.NS       VINNY.NS       VINNY.NS   
Date                                                                    
2023-02-02 00:00:00+05:30    221.439619              1     221.439619   
2023-02-03 00:00:00+05:30    225.550904              1     225.550904   

                          SUPERTs_12_3.0      EMA_20                          
                                VINNY.NS    VINNY.NS    VINNY.NS    VINNY.NS  
Date                                                                          
2023-02-02 00:00:00+05:30            NaN  209.058629  210.977903  200.282338  
2023-02-03 00:00:00+05:30            NaN  214.881618  217.218103  205.417353  

[2 rows x 1664 columns]